In [1]:
!pip install requests

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip3.9 install beautifulsoup4

     |████████████████████████████████| 115 kB 784 kB/s eta 0:00:01


In [78]:
import requests
from bs4 import BeautifulSoup as bs


In [79]:
# 2.1
def get_links():  
    URL = "https://www.cian.ru/kupit-1-komnatnuyu-kvartiru/"
    res = requests.get(URL)
    soup = bs(res.content, 'html')
    items = [item for item in soup.find_all("div", {'class' : "_93444fe79c--general--2SDGY"}, limit=10)]
    links = [item.a.get('href') for item in items]
    return links
links = get_links()

In [81]:
#Общая стоимость, адрес, метро, тип жилья,| площадь комнат, Этаж, Срок сдачи
import time
from tqdm import tqdm

def get_inf(links):
    flats = []
    data=""
    for link in tqdm(links):
        res =  requests.get(link)
        soup = bs(res.content, 'html')
        cost = soup.find("span", {'itemprop' : "price"}).text.strip()
        cost = "".join(cost.split())[:-1]
        address = soup.find("address",{"class":"a10a3f92e9--address--140Ec"}).text[:-8]
        under = soup.find("a", {'class' : "a10a3f92e9--underground_link--AzxRC"}).text.strip()
        typ = soup.find("span", {"class":"a10a3f92e9--value--3Ftu5"}).text.strip()
        s = soup.find_all("div", {"class":"a10a3f92e9--info--3XiXi"})
        floor = ''
        sq = ''
        t = ''
        for i in s:
            i = str(i)
            if 'Общая' in i:
                sq = i[i.find('18c8R')+len('18c8R')+2:i.find('</div><div class="a10a3f92e9--info-title--2bXM9')]
            elif 'Жилая' in i:
                sq = i[i.find('18c8R')+len('18c8R')+2:i.find('</div><div class="a10a3f92e9--info-title--2bXM9')]
            elif 'Этаж' in i:
                floor = i[i.find('18c8R')+len('18c8R')+2:i.find('</div><div class="a10a3f92e9--info-title--2bXM9')]
            elif 'Срок сдачи' in i:
                t = i[i.find('18c8R')+len('18c8R')+2:i.find('</div><div class="a10a3f92e9--info-title--2bXM9')]
        flats.append([cost, address, under, typ, sq, floor, t])
        time.sleep(5)
    return flats
flats = get_inf(links)
        

100%|██████████| 10/10 [01:01<00:00,  6.18s/it]


In [80]:
links

['https://www.cian.ru/sale/flat/245541181/',
 'https://www.cian.ru/sale/flat/227192015/',
 'https://www.cian.ru/sale/flat/246717609/',
 'https://www.cian.ru/sale/flat/246614666/',
 'https://www.cian.ru/sale/flat/244876598/',
 'https://www.cian.ru/sale/flat/244751681/',
 'https://www.cian.ru/sale/flat/244703533/',
 'https://www.cian.ru/sale/flat/246868084/',
 'https://www.cian.ru/sale/flat/245986062/',
 'https://www.cian.ru/sale/flat/242213434/']

In [82]:
flats

[['21272737',
  'Москва, ЦАО, р-н Басманный, Бакунинская ул., 5',
  'Бауманская',
  'Новостройка Апартаменты',
  '55,3\xa0м²',
  '9 из 12',
  '4 кв. 2020'],
 ['32772547',
  'Москва, ЦАО, р-н Арбат, ул. Новый Арбат, 15',
  'Арбатская',
  'Новостройка Апартаменты',
  '53,9\xa0м²',
  '12 из 26',
  '2 кв. 2020'],
 ['15900000',
  'Москва, СВАО, р-н Лосиноостровский, Янтарный проезд, 9',
  'Свиблово',
  'Вторичка',
  '57\xa0м²',
  '7 из 16',
  ''],
 ['55335000',
  'Москва, ЦАО, р-н Арбат, ул. Арбат, 39',
  'Смоленская',
  'Новостройка',
  '65,1\xa0м²',
  '2 из 5',
  '4 кв. 2018'],
 ['10609435',
  'Москва, СЗАО, р-н Покровское-Стрешнево, Волоколамское ш.',
  'Трикотажная',
  'Новостройка',
  '13,2\xa0м²',
  '18 из 28',
  '4 кв. 2022'],
 ['14070388',
  'Москва, СЗАО, р-н Хорошево-Мневники, Причальный проезд, 8',
  'Шелепиха',
  'Новостройка',
  '35\xa0м²',
  '29 из 32',
  '4 кв. 2023'],
 ['61080000',
  'Москва, ЦАО, р-н Мещанский, пер. Печатников, 19С1',
  'Трубная',
  'Вторичка',
  '101,8\xa0

In [83]:
flats2 = flats
for i in flats2:
    i[0] = int(i[0])
    try:
        i[4] = float(i[4].replace('\xa0м²', '').replace(',', '.'))
    except:
        pass
    i[5] = i[5].replace(' из ', '/')
    
import pandas as pd
df = pd.DataFrame(np.array(flats2), columns=['Цена', 'Адрес','Метро','Тип', 'Площадь', 'Этаж','Срок сдачи'])
for i in range(len(df['Цена'])):
    df['Цена'][i] = int(df['Цена'][i])
    df['Площадь'][i] = float(df['Площадь'][i])
df

,Цена,Адрес,Метро,Тип,Площадь,Этаж,Срок сдачи
0,21272737,"Москва, ЦАО, р-н Басманный, Бакунинская ул., 5",Бауманская,Новостройка Апартаменты,55.3,9/12,4 кв. 2020
1,32772547,"Москва, ЦАО, р-н Арбат, ул. Новый Арбат, 15",Арбатская,Новостройка Апартаменты,53.9,12/26,2 кв. 2020
2,15900000,"Москва, СВАО, р-н Лосиноостровский, Янтарный п...",Свиблово,Вторичка,57,7/16,
3,55335000,"Москва, ЦАО, р-н Арбат, ул. Арбат, 39",Смоленская,Новостройка,65.1,2/5,4 кв. 2018
4,10609435,"Москва, СЗАО, р-н Покровское-Стрешнево, Волоко...",Трикотажная,Новостройка,13.2,18/28,4 кв. 2022
5,14070388,"Москва, СЗАО, р-н Хорошево-Мневники, Причальны...",Шелепиха,Новостройка,35,29/32,4 кв. 2023
6,61080000,"Москва, ЦАО, р-н Мещанский, пер. Печатников, 19С1",Трубная,Вторичка,101.8,5/5,
7,9500000,"Москва, ЮЗАО, р-н Котловка, ул. Ремизова, 15К3",Нагорная,Вторичка,17,14/16,
8,7089000,"Москва, ЮВАО, р-н Кузьминки, Волгоградский просп.",Волгоградский проспект,Новостройка,25.5,3/30,1 кв. 2023
9,8300000,"Москва, ЗАО, р-н Можайский, Сколковское ш., 30",Сетунь,Вторичка,19.9,14/22,


In [84]:
df.loc[df['Цена']<=20000000].loc[df['Площадь']>=15].loc[df['Тип'] == 'Новостройка']\
.sort_values(by=['Цена'],ascending=False).head(10)

,Цена,Адрес,Метро,Тип,Площадь,Этаж,Срок сдачи
5,14070388,"Москва, СЗАО, р-н Хорошево-Мневники, Причальны...",Шелепиха,Новостройка,35,29/32,4 кв. 2023
8,7089000,"Москва, ЮВАО, р-н Кузьминки, Волгоградский просп.",Волгоградский проспект,Новостройка,25.5,3/30,1 кв. 2023
